# Finder chart with parallax and proper motion

And possibly the depiction of the orbit.

In [ ]:
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import exoplanet as xo

from astropy import constants

# conversion constant from au to R_sun
au_to_R_sun = (constants.au / constants.R_sun).value

We take orbital parameters for Luhman 16 from Table 5 of [Lazorenko & Sahlmann 2018](https://ui.adsabs.harvard.edu/abs/2018A%26A...618A.111L/abstract):

| Param | Value|
|-- | -- |
|$a$ (mas)			| 1784.0|
|$a$ (AU)			| 3.557|
|$e$			      | 0.343 |
|$P$ (yr)			| 27.54|
|$T_0$ (yr)			| 2017.78|
|$i$ (deg)			| 100.26|
|$\omega$ (deg)			| 128.1|
|$\Omega$ (deg)			| 139.67|
|$M_{tot}$ ($M_{\mathrm{Jup} }$)	| 		62.06|
|$M_A$ ($M_{\mathrm{Jup} }$)		| 	33.51|
|$M_B$ ($M_{\mathrm{Jup} }$)		| 	28.55|

In [ ]:
parallax = 501.557 # mas

In [ ]:
a = (3.557 * u.AU).to(u.Rsun).value
e = 0.343
i = 100.26 * np.pi / 180  # [rad]
omega = 128.1 * np.pi / 180  # Pourbaix reports omega_2, but we want omega_1
Omega = 139.67 * np.pi / 180
P = 27.54 * 365.25  # days

T0_orig = Time(2017.78, format="decimalyear")
T0_orig.format = "jd"
T0 = T0_orig.value  # [Julian Date]

In [ ]:
# instantiate the orbit
orbit = xo.orbits.KeplerianOrbit(
    a=a, t_periastron=T0, incl=i, ecc=e, omega=omega, Omega=Omega, 
    m_planet=0.027, m_star=0.032, m_planet_units=u.Msun, r_star = 0.10045
)

In [ ]:
import theano
import theano.tensor as tt

In [ ]:
t_arr = ( T0_orig - P/2*u.day) + np.linspace(0*u.day, P*u.day, num=200)  # days

In [ ]:
t = np.linspace(T0 - P/2, T0 + P/2, num=200)  # days
rho, theta = theano.function([], orbit.get_relative_angles(t, parallax))()

In [ ]:
current_jd = Time.now().jd
current_decyear = Time.now().decimalyear

In [ ]:
rho_now, theta_now = theano.function([], orbit.get_relative_angles(current_jd, parallax))()

In [ ]:
Thst_i = Time(2014.64, format="decimalyear").jd
Thst_f = Time(2016.76, format="decimalyear").jd

In [ ]:
t_hst = np.linspace(Thst_i,Thst_f, num=200)  # days
rho_hst, theta_hst = theano.function([], orbit.get_relative_angles(t_hst, parallax))()

In [ ]:
#import seaborn as sns

In [ ]:
#sns.set_style('darkgrid')
#sns.set_context('talk')

In [ ]:
# Plot the orbit
fig, ax = plt.subplots(nrows=1, figsize=(6, 8))

xs = rho * np.cos(theta)  # X is north
ys = rho * np.sin(theta)  # Y is east
plt.scatter([0], [0], marker='*', s=400, c='#2980b9', ec='k')
ax.plot(ys, xs, lw=2, color='#e67e22')



xnow = rho_now * np.cos(theta_now)  # X is north
ynow = rho_now * np.sin(theta_now)  # Y is east

xhst = rho_hst * np.cos(theta_hst)  # X is north
yhst = rho_hst * np.sin(theta_hst)  # Y is east

ax.plot(yhst, xhst, lw=8, color='#34495e', zorder=1)


label = '{:0.2f}'.format(current_decyear)
plt.scatter([ynow], [xnow], marker='s', s=100, c='#2ecc71', ec='k', 
            label = label, zorder=10)

plt.text(ynow-50, xnow-200, label)

plt.ylim(-1500, 2000)
plt.xlim(1000, -1500)
plt.xlabel(r'$\Delta \alpha \cos \delta$ (mas)')
plt.ylabel(r'$\Delta \delta $ (mas)')
plt.title('Luhman 16 for IGRINS DDT')
plt.savefig('../../figures/Luhman16_orbit_demo.png', bbox_inches='tight', dpi=300)

What are the separation and PA right now?

In [ ]:
rho_now

In [ ]:
theta_now * 180/np.pi

## Position on the sky

In [ ]:
Dt_arr = t_arr - t_arr[0]  # days

In [ ]:
Dt_arr.to(u.year)

In [ ]:
compA_coords = theano.function([], orbit.get_star_position(t, parallax))()
compB_coords = theano.function([], orbit.get_planet_position(t, parallax))()

In [ ]:
xA, yA, zA = compA_coords
xB, yB, zB = compB_coords

In [ ]:
plt.plot(xA, yA)
plt.plot(xB, yB)

## Compute the parallax and proper motion

Compute the parallax factors indirectly see this astropy Issue:  
https://github.com/astropy/astropy/issues/9140

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
distance = 1.0/0.50114 * u.pc

In [ ]:
c = SkyCoord(ra=96.9342078*u.degree /np.cos(53.3179180*np.pi/180), 
             dec=-53.3179180*u.degree, 
             distance=distance, 
             pm_ra_cosdec=-2763*u.mas/u.yr,
             pm_dec=+358*u.mas/u.yr, 
             radial_velocity=0.0*u.km/u.s,
             frame='icrs',
             obstime=T0_orig
            )

In [ ]:
c_infinity = SkyCoord(ra=96.9342078*u.degree/np.cos(53.3179180*np.pi/180), 
             dec=-53.3179180*u.degree, 
             distance=1e9*u.pc, 
             pm_ra_cosdec=0.0*u.mas/u.yr,
             pm_dec=0.0*u.mas/u.yr, 
             radial_velocity=0.0*u.km/u.s,
             frame='icrs',
             obstime=T0_orig
            )

In [ ]:
#cluh = SkyCoord.from_name('Luhman 16')

In [ ]:
#now = Time.now()

In [ ]:
coords = c.apply_space_motion(dt=Dt_arr) 
coords_gcrs = coords.transform_to('gcrs')

In [ ]:
ra_t   = coords_gcrs.ra
dec_t  = coords_gcrs.dec
cosd_t = np.cos(dec_t.to('radian'))

In [ ]:
coord_inf = c_infinity.apply_space_motion(dt=Dt_arr)
coord_inf_gcrs = coord_inf.transform_to('gcrs')

In [ ]:
ra0_t = coord_inf_gcrs.ra
dec0_t = coord_inf_gcrs.dec

dra = ((ra_t - ra0_t) * cosd_t).to('arcsec')  # in arcsec
ddec = (dec_t - dec0_t).to('arcsec')          # in arcsec

In [ ]:
plt.plot(c.ra+dra+xA*u.mas, c.dec+ddec+yA*u.mas)
plt.plot(c.ra+dra+xB*u.mas, c.dec+ddec+yB*u.mas)

## Overlay on a 2MASS finder chart

In [ ]:
from astroplan.plots import plot_finder_image
from astroplan import FixedTarget
import matplotlib.pyplot as plt

In [ ]:
targ = FixedTarget.from_name("Luhman 16")

from astroquery.skyview import SkyView; 

#SkyView.list_surveys()

In [ ]:
c_TESS = SkyCoord(ra=162.303282427*u.degree, 
             dec=-53.317573814*u.degree, 
             distance=distance, 
             pm_ra_cosdec=-2763*u.mas/u.yr,
             pm_dec=+358*u.mas/u.yr, 
             radial_velocity=20.0*u.km/u.s,
             frame='icrs',
             obstime=Time('2019.26', format='decimalyear')
            )

In [ ]:
from astroquery.vizier import Vizier

from astropy.coordinates import Angle

Vizier.ROW_LIMIT = -1
result = Vizier.query_region(
    targ.coord,
    catalog=["I/345/gaia2"],
    radius=Angle(600, "arcsec"),
)

result = result["I/345/gaia2"].to_pandas()
result = result[result.Gmag < 14]

# Apply correction for proper motion
year = ((current_jd - 2457206.375) * u.day).to(u.year)
pmra = (
    ((np.nan_to_num(np.asarray(result.pmRA)) * u.milliarcsecond / u.year) * year)
    .to(u.deg)
    .value
)
pmdec = (
    ((np.nan_to_num(np.asarray(result.pmDE)) * u.milliarcsecond / u.year) * year)
    .to(u.deg)
    .value
)
result.RA_ICRS += pmra
result.DE_ICRS += pmdec

# Gently size the points by their Gaia magnitude
result['sizes'] = 64.0 / 2 ** (result["Gmag"] / 5.0)

In [ ]:
! ls ../data/HST

In [ ]:
from astropy.io import fits

In [ ]:
from astropy.wcs import WCS

In [ ]:
plt.figure(figsize=(10,10))
ax, hdu = plot_finder_image(targ, survey='2MASS-H', log=True, reticle=True, grid=True, fov_radius=5*u.arcmin)

cos_dec = np.cos(c.dec.to(u.radian))
net_ra = c.ra+dra+xA*u.mas
net_raB = c.ra+dra+xB*u.mas

#ax.scatter(net_ra, c.dec+ddec+yA*u.mas, transform=ax.get_transform('icrs'), s=1)
#ax.scatter(net_raB, c.dec+ddec+yB*u.mas, transform=ax.get_transform('icrs'), s=1)
ax.scatter(c_TESS.ra, c_TESS.dec, transform=ax.get_transform('icrs'), s=500, marker='*')

ax.scatter(result.RA_ICRS.values, result.DE_ICRS.values, s=result.sizes.values*8, transform=ax.get_transform('icrs'), 
           alpha=0.6, fc='none', ec='r')

ax.set_xlim(50, 250)
ax.set_ylim(50,250)

Too zoomed out.  Let's try the HST data!

In [ ]:
from astropy.nddata import Cutout2D

The HST data fails because the header is malformed!  Womp womp.

```python
hdu = fits.open('../data/HST/sumF814W_gaiaWCS.fits/sumF814W_gaiaWCS.fits')[0]
wcs = WCS(hdu.header)
cutout = Cutout2D(hdu.data, targ.coord, (10,10), wcs=wcs)
```

fail!

Let's try the TESS data, which is coarse but standardized.

In [ ]:
import lightkurve as lk

In [ ]:
sr = lk.search_tesscut('Luhman 16', )

In [ ]:
tpf = sr.download(cutout_size=(21, 41))

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.subplot(projection=tpf.wcs, label='overlays')
ax.imshow(tpf.flux[300].value, vmin=100, vmax=300, origin='lower')

cos_dec = np.cos(c.dec.to(u.radian))
net_ra = c.ra+dra+xA*u.mas
net_raB = c.ra+dra+xB*u.mas

#ax.scatter(net_ra/cos_dec, c.dec+ddec+yA*u.mas, transform=ax.get_transform('icrs'), s=1)
#ax.scatter(net_raB/cos_dec, c.dec+ddec+yB*u.mas, transform=ax.get_transform('icrs'), s=1)


ax.scatter(result.RA_ICRS.values, result.DE_ICRS.values, s=result.sizes.values*8, transform=ax.get_transform('icrs'), 
           alpha=0.6, fc='none', ec='r')

ax.scatter(c_TESS.ra, c_TESS.dec, s=400, marker='*',transform=ax.get_transform('icrs'), 
           alpha=1, fc='w', ec='k')

ax.set_xlim(0, 40)
ax.set_ylim(2, 18)

In [ ]:
tpf.interact()

In [ ]:
lk.__version__